In [ ]:
import aiohttp
import asyncio
from autogen_agentchat.agents import AssistantAgent

class SearchAgent(AssistantAgent):
    """
    Agent that performs factual web lookups using DuckDuckGo and Wikipedia,
    then summarizes the result using an LLM (model_client).
    """

    def __init__(self, name="search", model_client=None):
        super().__init__(
            name=name,
            model_client=model_client,
            system_message=(
                "You are the SearchAgent. Given a query, you fetch factual information from the DuckDuckGo "
                "Instant Answer API and Wikipedia, then generate a natural and concise summary using the LLM. "
                "Avoid simulating sources. Prioritize real-world facts."
            )
        )

    async def search(self, query: str) -> str:
        cleaned_query = self._simplify_query(query)
        ddg_result, wiki_result = await self._run_parallel_searches(cleaned_query, query)

        if not ddg_result and not wiki_result:
            return f"ℹ️ No useful information found for \"{query}\" from either DuckDuckGo or Wikipedia."

        summary = await self._summarize_with_llm(ddg_result, wiki_result)
        return f"📘 Summary:\n{summary}"

    async def _run_parallel_searches(self, ddg_query: str, wiki_query: str):
        return await asyncio.gather(
            self._search_duckduckgo(ddg_query),
            self._search_wikipedia(wiki_query)
        )

    async def _search_duckduckgo(self, query: str) -> str:
        url = "https://api.duckduckgo.com/"
        params = {
            "q": query,
            "format": "json",
            "no_redirect": 1,
            "no_html": 1,
            "skip_disambig": 1
        }
        try:
            async with aiohttp.ClientSession(trust_env=True) as sess:
                async with sess.get(url, params=params, timeout=5) as resp:
                    data = await resp.json(content_type=None)
            return data.get("AbstractText", "").strip()
        except Exception:
            return ""

    async def _search_wikipedia(self, query: str) -> str:
        url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{query.replace(' ', '_')}"
        try:
            async with aiohttp.ClientSession(trust_env=True) as sess:
                async with sess.get(url, timeout=5) as resp:
                    if resp.status != 200:
                        return ""
                    data = await resp.json()
                    return data.get("extract", "").strip()
        except Exception:
            return ""

    async def _summarize_with_llm(self, ddg_text: str, wiki_text: str) -> str:
        """
        Use LLM (model_client) to summarize and deduplicate DDG and Wikipedia responses.
        """
        prompt = (
            "You are a smart summarizer. Given two factual text sources about the same topic, "
            "summarize them into a single clean, natural-sounding paragraph without repetition. "
            "If both sources are similar, merge the information without duplication. "
            "Avoid mentioning the sources directly.\n\n"
            f"Source 1 (DuckDuckGo):\n{ddg_text or '[empty]'}\n\n"
            f"Source 2 (Wikipedia):\n{wiki_text or '[empty]'}\n\n"
            "Final Summary:"
        )
        try:
            response = await self.model_client.aask(prompt)
            return response.strip()
        except Exception as e:
            return f"❌ LLM summarization failed: {e}"

    def _simplify_query(self, query: str) -> str:
        """
        Optionally simplify/clean the user query for use in search APIs.
        """
        return query.strip().split(" and ")[0].split("?")[0].strip()
